## Setting Up and Importing Libraries
In this step, we'll import the necessary libraries and modules. We'll also ensure the correct path is set to access our custom modules.

In [1]:
# Import necessary libraries
import pandas as pd
import os
import sys
import json

os.chdir(os.path.join(os.getcwd(), '..'))
print(os.getcwd())

from src.cfsearch import CFsearch
from src.dataset import Dataset
from src.explainable_model import ExplainableModel
from src.ceinstance.instance_sampler import CEInstanceSampler
from src.config import Config
from src.transformer import Transformer
from src.ceinstance.instance_factory import InstanceFactory

/home/rita/TRUST_AI/trustframework/trustCE


## Loading Configuration
Here, we'll load our configuration files which dictate various parameters for our counterfactual search. It includes dataset details, feature management, and other related configurations.

In [2]:
# Load configuration
config_file_path = "config/conf.yaml"
config = Config(config_file_path)

with open("config/constraints_conf.json", 'r') as file:
    constraints = json.load(file)

print("Configuration Loaded:")
print(config)

Configuration Loaded:


## Preparing Dataset and Model
In this section, we initialize our dataset, model, and the required transformers. We'll also define a sample instance for which we wish to find the counterfactuals.

In [3]:
# Set the target instance path
target_instance_json = "datasets/instance.json"

# Load the dataset and set up the necessary objects
data = Dataset(config.get_config_value("dataset"), "Loan_Status")
normalization_transformer = Transformer(data, config.get_config_value("feature_manager"))
instance_factory = InstanceFactory(data)
sampler = CEInstanceSampler(config, normalization_transformer, instance_factory)

model = ExplainableModel(config.get_config_value("model"))

Features verified
Continious features: ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']
Categorical features: ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']
Dataset preprocessed
Feature: CoapplicantIncome
Range: [-0.548056854219573, 13.372167288446008]
Feature: Credit_History
Range: [-2.511080956910919, 0.3975458829578409]
Feature: Self_Employed
Range: [0, 1]
Feature: Married
Range: [0, 1]
Feature: Property_Area
Range: [0, 1]
Feature: Loan_Amount_Term
Range: [-5.044846090672854, 2.106513522957348]
Feature: LoanAmount
Range: [-1.5999485916282457, 6.4030605082645256]
Feature: ApplicantIncome
Range: [-0.8484208485011342, 12.13039262846177]
Feature: Dependents
Range: [0, 1]
Feature: Education
Range: [0, 1]
Feature: Gender
Range: [0, 1]
Constraint Type: immutable
Sanity check for model
Model input shape is  11


/home/rita/TRUST_AI/trustframework/trustCE/trustce/lib/python3.9/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/rita/TRUST_AI/trustframework/trustCE/trustce/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


## Finding Counterfactuals
With everything set up, we'll now search for counterfactuals for our sample instance using the CFsearch object.

In [4]:
# Create a CFsearch object
config_for_cfsearch = config.get_config_value("cfsearch")
search = CFsearch(normalization_transformer, model, sampler, algorithm=config_for_cfsearch["optimizer"], 
                  distance_continuous=config_for_cfsearch["continuous_distance"], 
                  distance_categorical=config_for_cfsearch["categorical_distance"], 
                  loss_type=config_for_cfsearch["loss_type"], 
                  coherence=config_for_cfsearch["coherence"],
                  objective_function_weights=config_for_cfsearch["objective_function_weights"])

# Load target instance and find counterfactuals
with open(target_instance_json, 'r') as file:
    target_instance_json_content = file.read()

target_instance = instance_factory.create_instance_from_json(target_instance_json_content)

In [5]:
counterfactuals = search.find_counterfactuals(target_instance, 1, "opposite", 50)

## Evaluation and Visualization
Once the counterfactuals are generated, it's crucial to evaluate and visualize them. This helps in understanding how the counterfactuals differ from the original instance and assessing their quality.

In [6]:
# Evaluate and visualize the counterfactuals
search.evaluate_counterfactuals(target_instance, counterfactuals)

# Display the counterfactuals and original instance in the notebook
display_df = search.visualize_as_dataframe(target_instance, counterfactuals)
display(display_df)

CF instance:  {'Gender': 1, 'Married': 0, 'Dependents': 1, 'Education': 1, 'Self_Employed': 0, 'Property_Area': 0, 'ApplicantIncome': 7.980840017818599, 'CoapplicantIncome': 6.137497592499862, 'LoanAmount': 2.618565137070548, 'Loan_Amount_Term': -2.7065572315492057, 'Credit_History': -0.8135212565679412}
Distance continuous:  22.892071122370716
Distance categorical:  3
Sparsity continuous:  5
Sparsity categorical:  3
Validity:  False
Query instance (original outcome : 0)


,Gender,Married,Dependents,Education,Self_Employed,Property_Area,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
0,1,1,1,1,1,1,-0.872309,-0.547016,-1.677108,-5.224045,-1.354768



Counterfactual set (new outcome: [0])


,Gender,Married,Dependents,Education,Self_Employed,Property_Area,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
0,-,0,-,-,0,0,7.980840017818599,6.137497592499862,2.618565137070548,-2.7065572315492057,-0.8135212565679412


None

## Storing the Results
For reproducibility and further analysis, we'll store the counterfactuals and their evaluations in designated folders.

In [7]:
# Store results
search.store_counterfactuals(config.get_config_value("output_folder"), "first_test")
search.store_evaluations(config.get_config_value("output_folder"), "first_eval")

# Print counterfactuals and original instance for clarity
print("Counterfactuals:")
print(counterfactuals[0].get_values_dict())
print("Original instance:")
print(target_instance.get_values_dict())

Counterfactuals:
{'Gender': 1, 'Married': 0, 'Dependents': 1, 'Education': 1, 'Self_Employed': 0, 'Property_Area': 0, 'ApplicantIncome': 7.980840017818599, 'CoapplicantIncome': 6.137497592499862, 'LoanAmount': 2.618565137070548, 'Loan_Amount_Term': -2.7065572315492057, 'Credit_History': -0.8135212565679412}
Original instance:
{'Gender': 1, 'Married': 1, 'Dependents': 1, 'Education': 1, 'Self_Employed': 1, 'Property_Area': 1, 'ApplicantIncome': -0.8723094327303865, 'CoapplicantIncome': -0.5470157982925149, 'LoanAmount': -1.6771080672486245, 'Loan_Amount_Term': -5.224045233166907, 'Credit_History': -1.3547683316604187}
